# Trabalho Final de Mineração de Dados

## Importando bibliotecas

In [1]:
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.dummy import DummyClassifier
from sklearn import metrics
from collections import Counter
import pandas as pd
import numpy as np
import nltk
import string
nltk.download("punkt")
nltk.download("stopwords")

C:\Users\demet\AppData\Roaming\Python\Python39\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\demet\AppData\Roaming\Python\Python39\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\demet\AppData\Roaming\Python\Python39\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\demet\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\demet\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## 1 - Fase de Preparação dos Dados
---

Carregando o arquivo original de classes.

In [2]:
classes_original = pd.read_excel("dados/SistemaDeClassificacao.xlsx", "Select viw_classificacao_arvore")
classes_original.head()

,,COD_CLASSE,DES_NOME_PREFERIDO,COD_CLASSE_PAI,NUM_NIVEL,CYCLE,TREE,PATH,QTD_FILHOS,DES_NIVEL1,DES_NIVEL2,DES_NIVEL3,DES_NIVEL4
0,1,33254494,Classificação Temática Unificada,NaN,1,0,Classificação Temática Unificada,Classificação Temática Unificada,202,NaN,NaN,NaN,NaN
1,2,33809814,Temas Exclusivos de Pronunciamentos,33254494.0,2,0,-- Temas Exclusivos de Pronunciamentos,Classificação Temática Unificada / Temas Exclu...,22,Temas Exclusivos de Pronunciamentos,NaN,NaN,NaN
2,3,33809634,Meio Ambiente,33254494.0,2,0,-- Meio Ambiente,Classificação Temática Unificada / Meio Ambiente,11,Meio Ambiente,NaN,NaN,NaN
3,4,33809514,"Soberania, Defesa Nacional e Ordem Pública",33254494.0,2,0,"-- Soberania, Defesa Nacional e Ordem Pública","Classificação Temática Unificada / Soberania, ...",7,"Soberania, Defesa Nacional e Ordem Pública",NaN,NaN,NaN
4,5,33808912,Política Social,33254494.0,2,0,-- Política Social,Classificação Temática Unificada / Política So...,37,Política Social,NaN,NaN,NaN


Filtra o arquivo de classes original para exibir apenas aquelas que serão raiz na classificação da ementa.
Essa classes possuem nível 2 na tabela original.

In [3]:
classes_raiz = classes_original.query("NUM_NIVEL == 2").filter(["COD_CLASSE", "DES_NOME_PREFERIDO"])
classes_raiz.rename(columns={"DES_NOME_PREFERIDO": "DES_CLASSE"}, inplace=True)
classes_raiz

,COD_CLASSE,DES_CLASSE
1,33809814,Temas Exclusivos de Pronunciamentos
2,33809634,Meio Ambiente
3,33809514,"Soberania, Defesa Nacional e Ordem Pública"
4,33808912,Política Social
5,33805362,Jurídico
6,33805317,Honorífico
7,33805137,Infraestrutura
8,33769167,Economia e Desenvolvimento
9,33768972,Organização do Estado
10,33685789,Administração Pública


Carrega o dataset contendo as leis que já foram classificadas. Com os dados carregados, cria uma coluna derivada (DES_CLASSE_RAIZ) a partir da árvore de classes (DES_CLASSE_HIERARQUIA), da qual a informação de classe raiz é extraída.

In [4]:
leis_classificadas_original = pd.read_excel("dados/ClassificacaoDeLeisOrdinarias-LeisComplementares-e-DecretosNumerados-Desde1900.xlsx", "Select mvw_u03_prc_doc_tema")
leis_classificadas_original["DES_CLASSE_RAIZ"] = leis_classificadas_original["DES_CLASSE_HIERARQUIA"].apply(lambda hierarquia : hierarquia.split(" / ")[1])
leis_classificadas_original.head()

,,COD_PRC_DOC_TEMA,COD_PROCESSO_DOCUMENTO,COD_CLASSE,DES_CLASSE,DES_CLASSE_HIERARQUIA,DES_CLASSE_RAIZ
0,1,36155183,386343,33805827,Crédito Suplementar,Classificação Temática Unificada / Orçamento P...,Orçamento Público
1,2,36192020,386579,33805287,Rádio e TV,Classificação Temática Unificada / Infraestrut...,Infraestrutura
2,3,36155185,387419,33805827,Crédito Suplementar,Classificação Temática Unificada / Orçamento P...,Orçamento Público
3,4,36192056,387832,33805287,Rádio e TV,Classificação Temática Unificada / Infraestrut...,Infraestrutura
4,5,36155187,388197,33805827,Crédito Suplementar,Classificação Temática Unificada / Orçamento P...,Orçamento Público


Carrega os dados de todas as leis (classificadas e não classificadas) e aplica as transformações iniciais.

In [5]:
leis_original = pd.read_excel("dados/LeisOrdinarias-LeisComplementare-e-DecretosNumeradosComClassificacaoDesde1900.xlsx", "Select mvw_s01_documento")
leis_original.rename(columns={"DBMS_LOB.SUBSTR(S01.TXT_EMENTA": "TXT_EMENTA"}, inplace=True)
leis_original.drop(columns="   ", inplace=True)
leis_original.head()

,COD_DOCUMENTO,DES_NOME_PREFERIDO,DES_NOMES_ALTERNATIVOS,TXT_EMENTA
0,35345364,Lei nº 14.263 de 22/12/2021,LEI-14263-2021-12-22,Abre ao Orçamento da Seguridade Social da Uniã...
1,26247104,Lei nº 13.486 de 03/10/2017,LEI-13486-2017-10-03,"Altera o art. 8º da Lei nº 8.078, de 11 de set..."
2,27445746,Lei nº 13.701 de 06/08/2018,LEI-13701-2018-08-06,Cria o cargo de natureza especial de Intervent...
3,36348502,Lei nº 14.447 de 09/09/2022,LEI-14447-2022-09-09,Altera os limites da Floresta Nacional de Bras...
4,32103727,Lei nº 13.988 de 14/04/2020,LEI-13988-2020-04-14,Dispõe sobre a transação nas hipóteses que esp...


In [6]:
leis_original.shape[0]

26959

Com os datasets necessários já carregados, cria-se um novo dataset a partir do dataset contendo todas as leis, incluindo-se a informação das classes contidadas nas leis do dataset de leis classificadas.

In [7]:
leis = leis_original.merge(leis_classificadas_original.filter(["COD_PROCESSO_DOCUMENTO","DES_CLASSE_RAIZ"]), left_on="COD_DOCUMENTO", right_on="COD_PROCESSO_DOCUMENTO", how="left")
leis = leis.merge(classes_raiz, left_on="DES_CLASSE_RAIZ", right_on="DES_CLASSE", how="left")
leis.drop(columns=["COD_PROCESSO_DOCUMENTO", "DES_CLASSE_RAIZ"], inplace=True)
leis.head()

,COD_DOCUMENTO,DES_NOME_PREFERIDO,DES_NOMES_ALTERNATIVOS,TXT_EMENTA,COD_CLASSE,DES_CLASSE
0,35345364,Lei nº 14.263 de 22/12/2021,LEI-14263-2021-12-22,Abre ao Orçamento da Seguridade Social da Uniã...,33260515.0,Orçamento Público
1,26247104,Lei nº 13.486 de 03/10/2017,LEI-13486-2017-10-03,"Altera o art. 8º da Lei nº 8.078, de 11 de set...",33808912.0,Política Social
2,27445746,Lei nº 13.701 de 06/08/2018,LEI-13701-2018-08-06,Cria o cargo de natureza especial de Intervent...,33768972.0,Organização do Estado
3,36348502,Lei nº 14.447 de 09/09/2022,LEI-14447-2022-09-09,Altera os limites da Floresta Nacional de Bras...,33685789.0,Administração Pública
4,36348502,Lei nº 14.447 de 09/09/2022,LEI-14447-2022-09-09,Altera os limites da Floresta Nacional de Bras...,33809634.0,Meio Ambiente


In [8]:
leis.shape[0]

27743

Limpa os registros duplicados de leis que possuem mais de uma classe folha, mas que tenham mesma classe raiz e exibe as leis que possuem mais de uma classe folha, mas que tenham classes raiz diferentes.

In [9]:
leis.drop_duplicates(inplace=True)
temp = leis[["COD_DOCUMENTO", "COD_CLASSE", "DES_CLASSE"]].groupby("COD_DOCUMENTO")
temp.filter(lambda x: len(x) > 1)

,COD_DOCUMENTO,COD_CLASSE,DES_CLASSE
3,36348502,33685789.0,Administração Pública
4,36348502,33809634.0,Meio Ambiente
5,36348502,33769167.0,Economia e Desenvolvimento
20,36062349,33808912.0,Política Social
21,36062349,33769167.0,Economia e Desenvolvimento
...,...,...,...
27349,36032872,33808912.0,Política Social
27350,35556312,33685789.0,Administração Pública
27351,35556312,33808912.0,Política Social
27359,35396946,33809514.0,"Soberania, Defesa Nacional e Ordem Pública"


Tamanho da base tratada de leis.

In [10]:
leis.shape[0]

27294

Total de classificações feitas em leis.

In [11]:
leis_classificadas = leis.query("not COD_CLASSE.isnull()")
quantidade_classificacoes = leis_classificadas.shape[0]
quantidade_classificacoes

17438

Número distinto de leis classificadas.

In [12]:
len(leis_classificadas["COD_DOCUMENTO"].unique())

17103

Quantidade de leis a classificar.

In [13]:
leis_nao_classificadas = leis.query("COD_CLASSE.isnull()")
leis_nao_classificadas.shape[0]

9856

Verificação da distribuição das classes (em %).

In [14]:
distribuicao_classes = leis_classificadas[["COD_CLASSE", "DES_CLASSE"]].groupby("DES_CLASSE").count() / quantidade_classificacoes * 100
distribuicao_classes.sort_values(by="COD_CLASSE", ascending=False)

,COD_CLASSE
DES_CLASSE,
Orçamento Público,64.181672
Infraestrutura,14.382383
Política Social,5.029246
Administração Pública,4.490194
Honorífico,3.847918
Economia e Desenvolvimento,3.693084
Jurídico,2.184884
"Soberania, Defesa Nacional e Ordem Pública",0.951944
Organização do Estado,0.745498


## Desbalanceamento

É possível verificar que há um desbalanceamento grande entre as classes, o que prejudica o processo de aprendizado. Vamos tentar atacá-lo utilizando a técnica SMOTE:

In [15]:
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from numpy import where


def printClasses(classes):
	counter = Counter(classes)
	for k,v in counter.items():
		per = v / len(classes) * 100
		print('Classe=%s, n=%d (%.3f%%)' % (k, v, per))

X_smote = leis_classificadas.drop(columns=["COD_CLASSE", "DES_CLASSE"]).copy()
y_smote = leis_classificadas["DES_CLASSE"].copy()
printClasses(y_smote)


Classe=Orçamento Público, n=11192 (64.182%)
Classe=Política Social, n=877 (5.029%)
Classe=Organização do Estado, n=130 (0.745%)
Classe=Administração Pública, n=783 (4.490%)
Classe=Meio Ambiente, n=86 (0.493%)
Classe=Economia e Desenvolvimento, n=644 (3.693%)
Classe=Honorífico, n=671 (3.848%)
Classe=Jurídico, n=381 (2.185%)
Classe=Infraestrutura, n=2508 (14.382%)
Classe=Soberania, Defesa Nacional e Ordem Pública, n=166 (0.952%)


## One Class Classifier

Montaremos um exemplo que possa trabalhar as classes separadamente, utilizando o OneClassSVM para predizer se um texto pertence a uma determinada classe de treinamento. No nosso caso, utilizaremos Administração Pública como o caso positivo, e os demais como negativos. O tutorial utilizado como base está aqui: https://www.kaggle.com/code/saikumar587/bbc-text-oneclass-classificaiton



### Preparação

Como Administração Pública será a nossa classe de treinamento, vamos adicionar, no nosso conjunto de treino, uma coluna para dizer se aquele registro faz parte da nossa classe-alvo. Se fizer, o valor armazenado será +1, caso contrário, -1.

A função abaixo tem esse objetivo.


In [29]:
def addPositiveNegativeMarker(dataFrame, columnName, classeAlvo):
    # resultDataFrame = dataFrame.copy()
    # mask = resultDataFrame['DES_CLASSE'].str.equals(classeAlvo)
    # resultDataFrame.loc[mask, columnName] = 1
    dataFrame[columnName] = np.where(dataFrame['DES_CLASSE'] == classeAlvo, 1, -1)

Vamos modificar agora o dataset de treino para incluir essa coluna:

In [30]:
MARKER_COLUMN_NAME = 'MARKER'
CLASSE_TREINAMENTO = 'Administração Pública'
addPositiveNegativeMarker(leis_classificadas, MARKER_COLUMN_NAME, CLASSE_TREINAMENTO)
leis_classificadas.head()

C:\Users\demet\AppData\Local\Temp\ipykernel_15008\907047299.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame[columnName] = np.where(dataFrame['DES_CLASSE'] == classeAlvo, 1, -1)


,COD_DOCUMENTO,DES_NOME_PREFERIDO,DES_NOMES_ALTERNATIVOS,TXT_EMENTA,COD_CLASSE,DES_CLASSE,MARKER
0,35345364,Lei nº 14.263 de 22/12/2021,LEI-14263-2021-12-22,Abre ao Orçamento da Seguridade Social da Uniã...,33260515.0,Orçamento Público,-1
1,26247104,Lei nº 13.486 de 03/10/2017,LEI-13486-2017-10-03,"Altera o art. 8º da Lei nº 8.078, de 11 de set...",33808912.0,Política Social,-1
2,27445746,Lei nº 13.701 de 06/08/2018,LEI-13701-2018-08-06,Cria o cargo de natureza especial de Intervent...,33768972.0,Organização do Estado,-1
3,36348502,Lei nº 14.447 de 09/09/2022,LEI-14447-2022-09-09,Altera os limites da Floresta Nacional de Bras...,33685789.0,Administração Pública,1
4,36348502,Lei nº 14.447 de 09/09/2022,LEI-14447-2022-09-09,Altera os limites da Floresta Nacional de Bras...,33809634.0,Meio Ambiente,-1


Criaremos agora um dataset que contém apenas os dados de Administração Pública:

In [31]:
administracaoPublica_df = leis_classificadas[leis_classificadas[MARKER_COLUMN_NAME] == 1]
administracaoPublica_df.shape

(783, 7)

Definiremos os datasets de treino e testes:

In [32]:
X = leis_classificadas.drop(columns=["COD_CLASSE", "DES_CLASSE"])
y = leis_classificadas[MARKER_COLUMN_NAME]

X_treino_dummy, X_teste, y_treino_dummy, y_teste = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

X_treino = administracaoPublica_df.drop(columns=["COD_CLASSE", "DES_CLASSE"])
y_treino = administracaoPublica_df[MARKER_COLUMN_NAME]


In [56]:
y_teste.head()

17925   -1
10308   -1
15421   -1
19450   -1
23970   -1
Name: MARKER, dtype: int32

## 2 - Estruturação do texto
***

Definição das funções para a tarefa de criação das matrizes de documentos (texto da ementa)

In [33]:
# Divide um texto em tokens utilizando a biblioteca de NLP nltk
def tokenizador(texto):
    texto = texto.lower()
    lista_alfanumerica = []

    for token_valido in nltk.word_tokenize(texto, language="portuguese"):
        # Se for caracter de pontuação ou for stopword
        if token_valido in string.punctuation:
            continue
        # elif token_valido in nltk.corpus.stopwords.words('portuguese'):
        #     continue

        lista_alfanumerica.append(token_valido)

    return lista_alfanumerica

In [34]:
# Combinação de vetores por soma
def combinacao_vetores(palavras_numeros, vocabulario):
    vetor_resultante = np.zeros(300)

    for pn in palavras_numeros:
        try:
            vetor_resultante += vocabulario.get_vector(pn)
        except KeyError:
            if pn.isnumeric():
                pn = "0" * len(pn) # "0, 00, 000, 0000, etc. dependendo do tamanho de caracteres do número
                vetor_resultante += vocabulario.get_vector(pn)
            else:
                vetor_resultante += vocabulario.get_vector("unknown")

    return vetor_resultante

In [35]:
def matriz_vetores(textos, vocabulario):
    x = len(textos)
    y = 300 # número de dimensões do vocabulário skip300
    matriz = np.zeros((x, y))

    for i in range(x):
        palavras_numeros = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_vetores(palavras_numeros, vocabulario)

    return matriz

Carrega o vocabulário Word Embeddings (Word2Vec - Skipgram) pré-treinado pelo NILC (Núcleo Interinstitucional de Linguística Computacional).

In [36]:
# Carrega vocabulário Skipgram
vocabulario_skipgram = KeyedVectors.load_word2vec_format("dados/skip_s300.txt")

In [68]:
from sklearn.base import TransformerMixin
def vetor(texto, vocabulario):
    y = 300 # número de dimensões do vocabulário skip300
    vetor = np.zeros((1, y))
    palavras_numeros = tokenizador(texto)
    vetor[0] = combinacao_vetores(palavras_numeros, vocabulario)
    return vetor

class Word2VecVectorizer(TransformerMixin):
    def transform(self, X, **transform_params):
        return matriz_vetores(X.TXT_EMENTA, vocabulario_skipgram)
    def fit(self, X, y=None, **fit_params):
        return self
    def get_params(self, deep=True):
            return {}
    
    

Criação da matriz de documentos do corpus (conjunto de ementas), aqui representada por matrizes por vetores (cada ementa é vetorizada assim como o vocabulário).

In [58]:
matriz_vetores_treino = matriz_vetores(X_treino["TXT_EMENTA"], vocabulario_skipgram)
matriz_vetores_teste = matriz_vetores(X_teste["TXT_EMENTA"], vocabulario_skipgram)

## 3 - Treinamento e predição do modelo de Machine Learning

Treinamento e predição do classificador One-class SVM

In [69]:
from sklearn.svm import OneClassSVM
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
# OneClassSVM algorithm
clf = OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
pipe = Pipeline([('vectorizer', Word2VecVectorizer()), ('oneClassSVM', clf)])
pipe.fit(X_treino, y_treino)

Pipeline(steps=[('vectorizer',
                 <__main__.Word2VecVectorizer object at 0x0000022CA342C190>),
                ('oneClassSVM', OneClassSVM(gamma=0.1, nu=0.1))])

In [71]:
# validate OneClassSVM model with train set
preds_train = pipe.predict(X_treino)

print("accuracy:", accuracy_score(y_treino, preds_train))

accuracy: 0.5006385696040868


In [73]:
preds_test = pipe.predict(X_teste)
preds_test

array([-1, -1, -1, ..., -1, -1, -1], dtype=int64)

In [74]:
from sklearn.metrics import confusion_matrix 
results = confusion_matrix(y_teste, preds_test) 
print('Confusion Matrix :')
print(results) 
print('Accuracy Score :',accuracy_score(y_teste, preds_test)) 
print('Report : ')
print(classification_report(y_teste, preds_test)) 

Confusion Matrix :
[[4976   21]
 [ 119  116]]
Accuracy Score : 0.9732415902140673
Report : 
              precision    recall  f1-score   support

          -1       0.98      1.00      0.99      4997
           1       0.85      0.49      0.62       235

    accuracy                           0.97      5232
   macro avg       0.91      0.74      0.80      5232
weighted avg       0.97      0.97      0.97      5232



In [75]:
X_teste.head()

,COD_DOCUMENTO,DES_NOME_PREFERIDO,DES_NOMES_ALTERNATIVOS,TXT_EMENTA,MARKER
17925,511791,Decreto nº 89.151 de 07/12/1983,DEC-89151-1983-12-07,ABRE A PRESIDENCIA DA REPUBLICA E AO SUBANEXO ...,-1
10308,511404,Decreto nº 88.764 de 26/09/1983,DEC-88764-1983-09-26,"ABRE AO MINISTERIO DO INTERIOR, EM FAVOR DE DI...",-1
15421,514754,Decreto nº 92.114 de 11/12/1985,DEC-92114-1985-12-11,"ABRE A PRESIDENCIA DA REPUBLICA, EM FAVOR DA S...",-1
19450,552504,Lei nº 10.628 de 24/12/2002,"LEI-10628-2002-12-24 , Lei do Foro Privilegiad...",Altera a redação do art. 84 do Decreto-Lei nº ...,-1
23970,484267,Decreto nº 61.642 de 03/11/1967,DEC-61642-1967-11-03,ABRE PELA PRESIDENCIA DA REPUBLICA EM FAVOR DO...,-1


In [76]:
testPositives = X_teste.query("MARKER==1")
testPositives.head()


,COD_DOCUMENTO,DES_NOME_PREFERIDO,DES_NOMES_ALTERNATIVOS,TXT_EMENTA,MARKER
2924,552209,Lei nº 10.333 de 19/12/2001,LEI-10333-2001-12-19,Extingue a 5ª e a 6ª Auditorias da 1ª Circunsc...,1
22521,575911,Lei nº 12.269 de 21/06/2010,LEI-12269-2010-06-21,Dispõe sobre o prazo para formalizar a opção p...,1
10065,34834674,Decreto nº 10.790 de 09/09/2021,DEC-10790-2021-09-09,"Altera o Decreto nº 9.783, de 7 de maio de 201...",1
6694,35699858,Decreto nº 11.051 de 26/04/2022,DEC-11051-2022-04-26,"Regulamenta o art. 24-D da Lei nº 9.636, de 15...",1
16671,588146,Lei nº 12.770 de 28/12/2012,LEI-12770-2012-12-28,Dispõe sobre o subsídio do Procurador-Geral da...,1


In [77]:
testNegatives = X_teste.query("MARKER==-1")
testNegatives.head()

,COD_DOCUMENTO,DES_NOME_PREFERIDO,DES_NOMES_ALTERNATIVOS,TXT_EMENTA,MARKER
17925,511791,Decreto nº 89.151 de 07/12/1983,DEC-89151-1983-12-07,ABRE A PRESIDENCIA DA REPUBLICA E AO SUBANEXO ...,-1
10308,511404,Decreto nº 88.764 de 26/09/1983,DEC-88764-1983-09-26,"ABRE AO MINISTERIO DO INTERIOR, EM FAVOR DE DI...",-1
15421,514754,Decreto nº 92.114 de 11/12/1985,DEC-92114-1985-12-11,"ABRE A PRESIDENCIA DA REPUBLICA, EM FAVOR DA S...",-1
19450,552504,Lei nº 10.628 de 24/12/2002,"LEI-10628-2002-12-24 , Lei do Foro Privilegiad...",Altera a redação do art. 84 do Decreto-Lei nº ...,-1
23970,484267,Decreto nº 61.642 de 03/11/1967,DEC-61642-1967-11-03,ABRE PELA PRESIDENCIA DA REPUBLICA EM FAVOR DO...,-1


In [47]:
preds_test[3]

-1